In [3]:
import time
from pprint import pprint
from itertools import *

import pandas as pd
pd.set_option('display.max_columns', None)
# import swifter

import spacy
nlp = spacy.load('en_core_web_lg')

In [4]:
# EntityTypes = ["PERSON", "NORP", "ORG", "FAC", "GPE", "LOC", "PRODUCT", "EVENT", "WORK_OF_ART"]

In [5]:
df = pd.read_csv(r"D:\Work\Coding\Data\some-news.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.drop("Unnamed: 0.1", axis=1)
df['article'] = df['article'].astype(str)

In [9]:
def getEnts(i):
    doc = nlp(i)
    entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}
    
    return entities

def addEnts(dataframe, columnName):

    extracted = []

    for i, row in dataframe.iterrows():
        ents = getEnts(row[columnName])
        extracted.append(ents)

    dataframe["ExtractedEnts"] = extracted
    
    return dataframe

def splitEnts(dataframe):
    df_temp = pd.json_normalize(dataframe['ExtractedEnts'])
    dataframe = pd.concat([dataframe, df_temp], axis=1)
    
    return dataframe

In [12]:
starttime = time.time()
# df2['new_text'] = df2.swifter.apply(lambda row: getEnts(row['article']), axis=1) 

df = addEnts(df, 'article')

print(f"{time.time() - starttime} seconds")

73.30217814445496 seconds


In [15]:
starttime = time.time()

df = splitEnts(df)

print(f"{time.time() - starttime} seconds")

0.040216684341430664 seconds


In [16]:
df

,date,year,month,day,author,title,article,url,section,publication,ExtractedEnts,CARDINAL,DATE,GPE,LOC,ORG,PERSON,WORK_OF_ART,MONEY,ORDINAL,PRODUCT,NORP,PERCENT,QUANTITY,TIME,FAC,EVENT,LANGUAGE,LAW
0,09-12-16 18:31,2016,12,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox,"{'CARDINAL': ['one'], 'DATE': ['30s', 'the pas...",[one],"[30s, the past two weeks, 30-something, the ne...",[Washington],[New America],"[the New York Times, New York Times, Journal o...","[Erik Voeten, Yascha Mounk, Polyarchy, Amanda ...",[Mounk and Foa],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07-10-16 21:26,2016,10,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider,"{'CARDINAL': ['32'], 'DATE': ['five-year', 'a ...",[32],"[five-year, a year ago, Friday, this year, las...",NaN,NaN,"[Colts, NFL, Denver Broncos, The Indianapolis ...","[Colts, Peyton Manning, Ryan Grigson, Luck, An...",NaN,"[$18.4 million, less than $1 million, $15 mill...",[30th],[Broncos],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26-01-18 00:00,2018,1,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters,"{'CARDINAL': ['four'], 'DATE': ['Friday', 'Thu...",[four],"[Friday, Thursday, last June]","[U.S., Switzerland]",NaN,"[DAVOS, Reuters, Davos, The New York Times, Wh...","[Steve Holland, Trump, Donald Trump, Robert Mu...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27-06-19 00:00,2019,6,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters,"{'CARDINAL': ['one'], 'DATE': ['This month', '...",[one],"[This month, 2017, May, 2007-2012, 2014, Thurs...","[PARIS, France]",NaN,"[Reuters, Macron, Le Point]","[Luke Baker, Carla Bruni, Simon Carraud, Franc...",NaN,NaN,[first],NaN,"[European, French]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27-01-16 00:00,2016,1,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ,"{'DATE': ['Wednesday', 'Sunday'], 'GPE': ['Par...",NaN,"[Wednesday, Sunday]","[Paris, Brinson, LAX, Switzerland]",NaN,NaN,"[Monty Brinson, Kim Richards, Monty, Paris Hil...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,20-07-18 00:00,2018,7,20,NaN,Christian Bale Shows Off Extreme Weight Loss A...,Christian Bale's ditched any vestige of Dick C...,https://www.tmz.com/2018/07/20/christian-bale-...,NaN,TMZ,"{'CARDINAL': ['one'], 'DATE': ['7 months ago',...",[one],"[7 months ago, months]",[Italy],NaN,"[CB, Christian Bale's]",[Dick Cheney],NaN,NaN,NaN,NaN,[Christian],NaN,"[about 228 pounds, roughly 100 lbs, almost 70 ...",[Friday morning],NaN,NaN,NaN,NaN
991,07-03-16 16:00,2016,3,7,Tim Shiel,THUMP Australia's Monthly Oceanic Report Will ...,The Oceanic Report is our new column bringing ...,https://www.vice.com/en_us/article/wnyx4w/thum...,Noisey,Vice,"{'CARDINAL': ['ten'], 'DATE': ['This month', '...",[ten],"[This month, 2016]","[New Zealand, Perth, Sydney, Melbourne]","[the Cook Islands, Rarotonga island, Swan Rive...","[NASA, Tourist Kid, SoundCloud, Twitter, Uni, ...","[Matthew Kirkis, Tim Shiel, Mei Saraswati, Kan...",[Cold Crush],NaN,NaN,[EP],"[Australian, Romanian]",NaN,NaN,NaN,NaN,[EP],NaN,NaN
992,20-12-16 00:00,2016,12,20,NaN,Nicole Kidman & Keith Urban Enjoy Christmas in...,Nicole Kidman and Keith Urban might have a whi...,https://www.tmz.com/2016/12/20/nicole-kidman-k...,NaN,TMZ,"{'DATE': ['the holidays', 'Monday', 'Christmas...",NaN,"[the holidays, Monday, Christmas, summer]",[

In [17]:
df.to_csv("Some News (NER Tagged).csv")